In [5]:
from utils.tf import *
from utils.db import *
from utils.gd2 import *
from utils.clt_repetidos import *
from datetime import datetime, time, timedelta
import cx_Oracle
import pandas as pd
import os
from utils.clt_repetidos import * 
from datetime import datetime
import re
import polars as pl
import gc
import polars as pl
from rapidfuzz import fuzz
from multiprocessing import Pool, cpu_count

In [6]:
def comparar_filas(args):
    idx1, nombre1, df2, col2, id2, umbral = args
    coincidencias = []

    for row in df2.iter_rows(named=True):
        nombre2 = row[col2]
        sim = fuzz.ratio(nombre1, nombre2)
        if sim >= umbral:
            coincidencias.append({
                "id_1": idx1,
                "nombre_1": nombre1,
                "id_2": row[id2],
                "nombre_2": nombre2,
                "score": sim
            })
    return coincidencias

def comparar_dataframes_polars(df1: pl.DataFrame, col1: str, id1: str,
                                df2: pl.DataFrame, col2: str, id2: str,
                                umbral: int = 90, n_procesos: int = None) -> pl.DataFrame:
    if n_procesos is None:
        n_procesos = max(cpu_count() - 1, 1)

    df2_rows = df2[[col2, id2]].clone()

    # Empaquetar argumentos para cada fila de df1
    argumentos = [
        (row[id1], row[col1], df2_rows, col2, id2, umbral)
        for row in df1.iter_rows(named=True)
    ]

    with Pool(n_procesos) as pool:
        resultados = pool.map(comparar_filas, argumentos)

    # Aplanar lista de listas
    coincidencias = [item for sublist in resultados for item in sublist]

    if not coincidencias:
        return pl.DataFrame()

    return pl.DataFrame(coincidencias)


# Pasar de excel a csv

In [ ]:
df_clientes_pld = pd.read_excel(
    "/home/ale1726/proyects/datalake/clientes/data/BASE ÚNICA DE CLIENTES 2.xlsx",
    sheet_name="CONSOLIDADO FINAL LIMPIO",
    header=1
)

In [ ]:
df_clientes_pld.to_csv(
    "/home/ale1726/proyects/datalake/clientes/data/BASE ÚNICA DE CLIENTES 2.csv",
    index=False,
    encoding="utf-8",
    sep=";"
)


# Comparación

In [7]:
df_clientes =  pl.read_csv("/home/ale1726/proyects/datalake/clientes/data/test/data/transformar/30_05_2025/CLientesT_FINAL_ACTIVOS.csv")

In [8]:

df_clientes_pld = pl.read_csv(
    "/home/ale1726/proyects/datalake/clientes/data/BASE ÚNICA DE CLIENTES 2.csv",
    encoding="utf8-lossy",             # tolerante a caracteres raros
    separator=";",                     # si usaste `sep=";"` al guardar
    ignore_errors=True,                # ignora filas malformadas
    truncate_ragged_lines=True         # completa filas cortadas con valores nulos
)


In [ ]:
df_clientes_pld.filter(col = "Número" )

In [14]:
df_80139 = df_clientes_pld.filter(pl.col("Número PLD") == "80139")

In [16]:
df_80139.select(pl.col("RFC").value_counts())


RFC
struct[2]
"{""FDA050428177"",1}"
"{""NFI3406305T0"",525}"
"{""-"",1}"


In [ ]:
df_80139

Dirección,Producto,Número Fiso,Número PLD,Cuenta/Contrato,Cliente,Intermediario,RFC,NOMBRE DE DIFM,NOMBRE FINAL,comentario
str,str,str,str,i64,str,str,str,str,str,str
"""DAPM""","""SERVICIOS DE INVERSIÓN""","""F80139""","""80139""",1063586,null,null,"""NFI3406305T0""",""" NAFIN FIDUCIARIO FIDEICOMISO …","""NACIONAL FINANCIERA SNC EN SU …",null
"""DAPM""","""SERVICIOS DE INVERSIÓN""","""F80139""","""80139""",1064792,null,null,"""NFI3406305T0""","""AFIN FISO CONTRAGTIA FIN EMP F…","""NACIONAL FINANCIERA SNC EN SU …",null
"""FISO""","""SERVICIOS FIDUCIARIOS""","""F80139""","""80139""",null,null,null,"""-""","""CONTRAGARANTIA PARA EL FINANCI…","""NACIONAL FINANCIERA SNC EN SU …",null
"""DAPM""","""SERVICIOS DE INVERSIÓN""","""F80139""","""80139""",1060824,null,null,"""NFI3406305T0""","""DIRECCION DE FIDEICOMISOS |FIS…","""NACIONAL FINANCIERA SNC EN SU …",null
"""DAPM""","""SERVICIOS DE INVERSIÓN""","""F80139""","""80139""",1062592,null,null,"""NFI3406305T0""","""DIRECCION DE FIDEICOMISOS 8013…","""DIRECCION DE FIDEICOMISOS 8013…",null
…,…,…,…,…,…,…,…,…,…,…
"""DAPM""","""SERVICIOS DE INVERSIÓN""","""F80139""","""80139""",1064767,null,null,"""NFI3406305T0""","""NAFIN SNC FIDU FISO CONTRAGARA…","""NACIONAL FINANCIERA SNC EN SU …",null
"""DAPM""","""SERVICIOS DE INVERSIÓN""","""F80139""","""80139""",1064791,null,null,"""NFI3406305T0""","""NAFIN, FISO CGTIA FINANCIAMIEN…","""NACIONAL FINANCIERA SNC EN SU …",null
"""DAPM""","""SERVICIOS DE INVERSIÓN""","""F80139""","""80139""",1064461,null,null,"""NFI3406305T0""","""NAFIN, S.N.C EN SU CARACTER DE…","""NACIONAL FINANCIERA SNC EN SU …",null


In [ ]:
df_clientes_pld.select(pl.col("Número PLD").unique())

Número PLD
str
"""10000352"""
"""80293"""
"""10000749"""
"""10000549"""
"""10000185"""
…
"""10000167"""
"""3017"""
"""10000926"""


In [ ]:
df_clientes_pld.unique("")

In [12]:
df_clientes_pld

Dirección,Producto,Número Fiso,Número PLD,Cuenta/Contrato,Cliente,Intermediario,RFC,NOMBRE DE DIFM,NOMBRE FINAL,comentario
str,str,str,str,i64,str,str,str,str,str,str
"""DPSYFC""","""PROGRAMA PRIMER PISO""","""-""","""10000009""",28594176,null,null,"""IEM020102LK6""","""-""","""EMERGING AMERICA INMOBILIARIA,…",null
"""FISO""","""SERVICIOS FIDUCIARIOS""","""F80449""","""80449""",null,null,null,"""FDA050428177""","""-""","""NAFIN FIDUCIARIO FIDEICOMISO A…",null
"""DAPM""","""SERVICIOS DE INVERSIÓN""","""F80139""","""80139""",1063586,null,null,"""NFI3406305T0""",""" NAFIN FIDUCIARIO FIDEICOMISO …","""NACIONAL FINANCIERA SNC EN SU …",null
"""DAPM""","""SERVICIOS DE INVERSIÓN""","""-""","""10000821""",1064707,null,null,"""STA130219T22""",""".+ TASA ACTIVO, S.A. DE C.V., …",""".+ TASA ACTIVO, S.A. DE C.V., …",null
"""DAPM""","""SERVICIOS DE INVERSIÓN""","""-""","""10000823""",1064327,null,null,"""STL100514QG9""",""".+TASA LARGO PLAZO, S.A. DE C.…",""".+TASA LARGO PLAZO, S.A. DE C.…",null
…,…,…,…,…,…,…,…,…,…,…
"""DAPM""","""SERVICIOS DE INVERSIÓN""","""-""","""10000860""",6009441,null,null,"""ZAM131014HV0""","""ZURICH FIANZAS MÉXICO, S.A. DE…","""ZURICH FIANZAS MÉXICO, S.A. DE…",null
"""DAPM""","""SERVICIOS DE INVERSIÓN""","""-""","""10000860""",1065073,null,null,"""ZAM131014HV0""","""ZURICH FIANZAS MÉXICO, S.A. DE…","""ZURICH FIANZAS MÉXICO, S.A. DE…",null
"""DAPM""","""SERVICIOS DE INVERSIÓN""","""-""","""10000816""",1064521,null,null,"""SSG971008QUI""","""ZURICH SANTANDER SEGUROS MÉXIC…","""ZURICH SANTANDER SEGUROS MEXIC…",null


In [11]:
resultado = comparar_dataframes_polars(
    df_clientes, "NOMBRE_O_RAZON_SOCIAL", "ID_DL",
    df_clientes_pld, "NOMBRE FINAL", "Número PLD",
    umbral=96
)

print(resultado)

shape: (1_424, 5)
┌──────┬─────────────────────────────────┬──────────┬─────────────────────────────────┬───────────┐
│ id_1 ┆ nombre_1                        ┆ id_2     ┆ nombre_2                        ┆ score     │
│ ---  ┆ ---                             ┆ ---      ┆ ---                             ┆ ---       │
│ i64  ┆ str                             ┆ str      ┆ str                             ┆ f64       │
╞══════╪═════════════════════════════════╪══════════╪═════════════════════════════════╪═══════════╡
│ 594  ┆ .+ TASA ACTIVO, S.A. DE C.V., … ┆ 10000821 ┆ .+ TASA ACTIVO, S.A. DE C.V., … ┆ 100.0     │
│ 595  ┆ .+TASA LARGO PLAZO, S.A. DE C.… ┆ 10000823 ┆ .+TASA LARGO PLAZO, S.A. DE C.… ┆ 100.0     │
│ 596  ┆ .+TASA PREVISIONAL, S.A. DE C.… ┆ 10000825 ┆ .+TASA PREVISIONAL, S.A. DE C.… ┆ 100.0     │
│ 597  ┆ .+TASA US, S.A. DE C.V., F.I.I… ┆ 10000828 ┆ .+TASA US, S.A. DE C.V., F.I.I… ┆ 100.0     │
│ 598  ┆ .+TASA, S.A. DE C.V., F.I.I.D.  ┆ 10000822 ┆ .+TASA, S.A. DE C.V., F.I.I.

In [10]:
df_joined = df_clientes.join(
    df_clientes_pld,
    left_on="NOMBRE_O_RAZON_SOCIAL",
    right_on="NOMBRE DE DIFM",
    how="inner"
)

In [51]:
df_joined

ID_DL,NEGOCIO,NOMBRE_O_RAZON_SOCIAL,NUMERO_CLIENTE,ESTATUS,NUMERO_CONTRATO,GENERO,FECHA_NAC_O_CONST,ENTIDAD_FEDERATIVA_NACIMIENTO,PAIS_DE_NACIMIENTO,NACIONALIDAD,PROFESION,CALLE,NUMERO_EXTERIOR,NUMERO_INTERIOR,COLONIA_URBANIZACION,DELEGACION_MUNICIPIO,CIUDAD_POBLACION,ENTIDAD_FEDERATIVA,CODIGO_POSTAL,PAIS,TELEFONO,CORREO_ELECTRONICO,RFC,CURP,FIEL,REPRESENTANTE_LEGAL,TIPO_PERSONA,PRODUCTO_CONTRATADO,SISTEMA_ORIGEN,FECHA_DE_ACTUALIZACION,map_ids,Dirección,Producto,Número Fiso,Número PLD,Cuenta/Contrato,Cliente,Intermediario,RFC_right,NOMBRE FINAL,comentario
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,str
600,"""Sin información""","""AAA HERFIN /10892""","""965201""","""VIGENTE""","""Sin información""","""NO APLICA""","""Sin información""","""Sin información""","""Sin información""","""MEXICANA""","""FIDEICOMISOS""","""Sin información""","""Sin información""","""Sin información""","""Sin información""","""Sin información""","""Sin información""","""Sin información""","""Sin información""","""MÉXICO""","""Sin información""","""Sin información""","""FAH931108RY9""","""Sin información""","""Sin información""","""OTERO HERNANDEZ JOSE MANUEL""","""MORAL NACIONAL GRAVABLE""","""Sin información""","""TAS""","""30/05/25""","""[TAS, 965201]""","""DAPM""","""SERVICIOS DE INVERSIÓN""","""F10892""","""10892""",965201,null,null,"""FAH931108RY9""","""AAA HERFIN | FISO No.10892""",null
601,"""Sin información""","""AAA LIVERPOOL /11786""","""1042271""","""VIGENTE""","""Sin información""","""NO APLICA""","""Sin información""","""Sin información""","""Sin información""","""MEXICANA""","""FIDEICOMISOS""","""Sin información""","""Sin información""","""Sin información""","""Sin información""","""Sin información""","""Sin información""","""Sin información""","""Sin información""","""MÉXICO""","""Sin información""","""Sin información""","""NFF990922JZ4""","""Sin información""","""Sin información""","""OTERO HERNANDEZ JOSE MANUEL""","""MORAL NACIONAL GRAVABLE""","""Sin información""","""TAS""","""30/05/25""","""[TAS, 1042271]""","""DAPM""","""SERVICIOS DE INVERSIÓN""","""F11786""","""11786""",1042271,"""N.A""","""N.A""","""NFF990922JZ4""","""AAA LIVERPOOL | FISO No.11786""",null
609,"""Sin información""","""ACE SEGUROS, S.A.""","""1064177, 6009249""","""VIGENTE""","""Sin información""","""NO APLICA""","""Sin información""","""Sin información""","""Sin información""","""MEXICANA""","""ASEGURADORAS""","""Sin información""","""Sin información""","""Sin información""","""Sin información""","""Sin información""","""Sin información""","""Sin información""","""Sin información""","""MÉXICO""","""Sin información""","""Sin información""","""ASE901221SM4""","""Sin información""","""Sin información""","""OCA�A RUIZ DE VELASCO ISABEL,…","""MORAL NACIONAL NO GRAVABLE""","""Sin información""","""TAS""","""30/05/25""","""[TAS, 1064177], [TAS, 6009249]""","""DAPM""","""SERVICIOS DE INVERSIÓN""","""-""","""10000170""",6009249,null,null,"""ASE901221SM4""","""ACE SEGUROS, S.A.""",null
610,"""Sin información""","""ACTICOBER, S.A. DE C.V. SOC D…","""1063229""","""VIGENTE""","""Sin información""","""NO APLICA""","""Sin información""","""Sin información""","""Sin información""","""MEXICANA""","""FONDOS DE INVERSION""","""Sin información""","""Sin información""","""Sin información""","""Sin información""","""Sin información""","""Sin información""","""Sin información""","""Sin información""","""MÉXICO""","""Sin información""","""Sin información""","""AII990517TT8""","""Sin información""","""Sin información""","""MADERO RIVERO ALVARO, MADERO R…","""MORAL NACIONAL NO GRAVABLE""","""Sin información""","""TAS""","""30/05/25""","""[TAS, 1063229]""","""DAPM""","""SERVICIOS DE INVERSIÓN""","""-""","""10000154""",1063229,null,null,"""AII990517TT8""","""ACTICOBER, S.A. DE C.V. SOC D…",null
611,"""Sin información""","""ACTICORP, S.A. DE C.V. SOC DE …","""1063225""","""V

In [48]:
print(df_clientes_pld.filter(pl.col("NOMBRE FINAL").str.contains("AXA SEGUROS SA DE CV")))


shape: (3, 11)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ Dirección ┆ Producto  ┆ Número    ┆ Número    ┆ … ┆ RFC       ┆ NOMBRE DE ┆ NOMBRE    ┆ comentar │
│ ---       ┆ ---       ┆ Fiso      ┆ PLD       ┆   ┆ ---       ┆ DIFM      ┆ FINAL     ┆ io       │
│ str       ┆ str       ┆ ---       ┆ ---       ┆   ┆ str       ┆ ---       ┆ ---       ┆ ---      │
│           ┆           ┆ str       ┆ str       ┆   ┆           ┆ str       ┆ str       ┆ str      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ DAPM      ┆ SERVICIOS ┆ -         ┆ 10000171  ┆ … ┆ ASE931116 ┆ AXA       ┆ AXA       ┆ null     │
│           ┆ DE        ┆           ┆           ┆   ┆ 231       ┆ SEGUROS   ┆ SEGUROS   ┆          │
│           ┆ INVERSIÓN ┆           ┆           ┆   ┆           ┆ SA DE CV  ┆ SA DE CV  ┆          │
│ DAPM      ┆ SERVICIOS ┆ -         ┆ 10000171  ┆ … ┆ ASE931116 ┆ AXA       

In [37]:
import polars as pl

# Simulamos un archivo CSV con datos
datos = pl.DataFrame({
    "cliente_id": [1, 2, 3, 4, 5],
    "pais": ["México", "Argentina", "México", "Chile", "México"],
    "ventas_2024": [1000, 2000, 1500, 1800, 1700],
})

# Guardamos los datos como si fuera un archivo real
datos.write_csv("clientes.csv")

In [46]:
# Leemos el archivo en modo lazy
df_lazy = pl.read_csv("clientes.csv").lazy()

In [47]:
df_lazy

In [45]:
del df_lazy

In [ ]:
# Creamos el pipeline: filtramos y seleccionamos columnas
resultado = (
    df_lazy
    .filter(pl.col("pais") == "México")
    .select(["cliente_id", "ventas_2024"])
    .collect()
)

In [43]:
# Imprimimos los resultados
print("Resultado (ya evaluado):")
print(resultado)

# Verificamos tipo para confirmar que está en memoria
print("\nTipo del resultado:", type(resultado))


Resultado (ya evaluado):
shape: (3, 2)
┌────────────┬─────────────┐
│ cliente_id ┆ ventas_2024 │
│ ---        ┆ ---         │
│ i64        ┆ i64         │
╞════════════╪═════════════╡
│ 1          ┆ 1000        │
│ 3          ┆ 1500        │
│ 5          ┆ 1700        │
└────────────┴─────────────┘

Tipo del resultado: <class 'polars.dataframe.frame.DataFrame'>
